# CLOSE AND OPEN EYE CLASSIFIER TRAINING

Once we have extract the eyes image from the original image we can use and train a classifier using a CNN for labeling eyes as open or close, using our dataset.

This first model has 4 convolutional layers, with 16*n kernels of size 3x3, an output layer with a sigmoid activation funcion for binary classification, and Adam optimizer. Also a reduced LR callback has been used every 5 epochs without improvement in the validation accuracy.

This model has 100% accuracy for train dataset, 97% for test dataset, and 95% for another public dataset download from kaggle.

There is a 2nd model with also 4 convolutional layers, but the 1st layer with less kernels applied, and the callback function for reducing LR has been change to 10 epochs without improvement. It has better accuracy for our dataset, but worse for kaggle dataset.

Finally, this models are saved to .h5 files, so they can be load and used in the eye detection code.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import os
import sklearn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
def load_images(root):
    #Method to load all the images and create X data and labels
    data = []
    for category in sorted(os.listdir(root)):
        for file in sorted(os.listdir(os.path.join(root, category))):
            data.append((category, os.path.join(root, category,  file)))

    df = pd.DataFrame(data, columns=['class', 'file_path'])
    df['class'] = LabelEncoder().fit_transform(df['class'].values)
    photos, labels = list(), list()
    for index,row in df.iterrows():
        photo = load_img(row['file_path'], target_size=(200, 200))
        photo = img_to_array(photo)
        photo = photo.astype("float32") / 255
        photos.append(photo)
        labels.append(row['class'])
    X = np.asarray(photos)
    y = np.asarray(labels)
    return X, y

In [3]:
class CustomCallbackReduceLr(keras.callbacks.Callback):
    def __init__(self):
        self.val_accuracy = np.Inf
        self.accumulator = 0
    def on_epoch_begin(self, epoch, logs={}):
        if self.accumulator == 5:
            self.model.optimizer.lr = self.model.optimizer.lr*0.5
            self.accumulator = 0
            print("Changing Learning Rate to ", float(keras.backend.get_value(self.model.optimizer.lr)))
    def on_epoch_end(self, epoch, logs={}):
        current = logs["val_accuracy"]
        if (current - self.val_accuracy) < 0.05:
            self.accumulator = self.accumulator + 1
            print("No change on val accuracy, increment accumulator to ", self.accumulator)
        else:
            self.accumulator = 0
        self.val_accuracy = current

In [4]:
class CustomCallbackReduceLr2(keras.callbacks.Callback):
    def __init__(self):
        self.val_accuracy = np.Inf
        self.accumulator = 0
    def on_epoch_begin(self, epoch, logs={}):
        if self.accumulator == 10:
            self.model.optimizer.lr = self.model.optimizer.lr*0.5
            self.accumulator = 0
            print("Changing Learning Rate to ", float(keras.backend.get_value(self.model.optimizer.lr)))
    def on_epoch_end(self, epoch, logs={}):
        current = logs["val_accuracy"]
        if (current - self.val_accuracy) < 0.05:
            self.accumulator = self.accumulator + 1
            print("No change on val accuracy, increment accumulator to ", self.accumulator)
        else:
            self.accumulator = 0
        self.val_accuracy = current

In [5]:
train_root = r'data\train'
test_root = r'data\test'
x_train, y_train = load_images(train_root)
x_test, y_test = load_images(test_root)

In [6]:
keras.backend.clear_session()
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu',padding='same', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 128)       7

In [6]:
change_lr_callback = CustomCallbackReduceLr()
history = model.fit(x_train, y_train, batch_size=10, epochs=50, verbose=1, validation_split=0.1, callbacks = [change_lr_callback])

Epoch 1/50
111/111 [==============================] - 14s 125ms/step - loss: 0.3310 - accuracy: 0.8541 - val_loss: 0.5164 - val_accuracy: 0.7984
Epoch 2/50
111/111 [==============================] - 15s 132ms/step - loss: 0.1796 - accuracy: 0.9333 - val_loss: 0.1059 - val_accuracy: 0.9435
Epoch 3/50
111/111 [==============================] - 16s 141ms/step - loss: 0.1281 - accuracy: 0.9559 - val_loss: 0.4046 - val_accuracy: 0.8145
Epoch 4/50
111/111 [==============================] - 16s 140ms/step - loss: 0.1583 - accuracy: 0.9468 - val_loss: 0.0411 - val_accuracy: 0.9839
Epoch 5/50
111/111 [==============================] - 16s 140ms/step - loss: 0.1066 - accuracy: 0.9604 - val_loss: 0.0493 - val_accuracy: 0.9758
Epoch 6/50
111/111 [==============================] - 16s 140ms/step - loss: 0.1119 - accuracy: 0.9631 - val_loss: 0.0930 - val_accuracy: 0.9597
Epoch 7/50
111/111 [==============================] - 15s 140ms/step - loss: 0.0732 - accuracy: 0.9703 - val_loss: 0.0551 - val_ac

Changing Learning Rate to  3.125000148429535e-05
Epoch 31/50
111/111 [==============================] - 16s 145ms/step - loss: 8.7821e-04 - accuracy: 1.0000 - val_loss: 0.2810 - val_accuracy: 0.9516
Epoch 32/50
111/111 [==============================] - 16s 145ms/step - loss: 8.6474e-04 - accuracy: 1.0000 - val_loss: 0.2716 - val_accuracy: 0.9516
Epoch 33/50
111/111 [==============================] - 16s 146ms/step - loss: 8.4176e-04 - accuracy: 1.0000 - val_loss: 0.2689 - val_accuracy: 0.9516
Epoch 34/50
111/111 [==============================] - 16s 146ms/step - loss: 8.2920e-04 - accuracy: 1.0000 - val_loss: 0.2760 - val_accuracy: 0.9516
Epoch 35/50
111/111 [==============================] - 16s 146ms/step - loss: 7.7363e-04 - accuracy: 1.0000 - val_loss: 0.2930 - val_accuracy: 0.9516
Changing Learning Rate to  1.5625000742147677e-05
Epoch 36/50
111/111 [==============================] - 16s 144ms/step - loss: 7.7115e-04 - accuracy: 1.0000 - val_loss: 0.2848 - val_accuracy: 0.9516
E

In [20]:
y_predicted = []
for i in range(len(y_test)):
    out_probabilities = model.predict(x_test[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    
a = model.evaluate(x_test, y_test)
print(classification_report(y_test, y_predicted))
print(confusion_matrix(y_test, y_predicted))

7/7 [==============================] - 1s 78ms/step - loss: 0.2356 - accuracy: 0.9587
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       109
           1       0.97      0.94      0.96       109

    accuracy                           0.96       218
   macro avg       0.96      0.96      0.96       218
weighted avg       0.96      0.96      0.96       218

[[106   3]
 [  6 103]]


In [21]:
y_predicted = []
for i in range(len(y_train)):
    out_probabilities = model.predict(x_train[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)

a = model.evaluate(x_train, y_train)
print(classification_report(y_train, y_predicted))
print(confusion_matrix(y_train, y_predicted))

39/39 [==============================] - 3s 82ms/step - loss: 0.0298 - accuracy: 0.9951
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       617
           1       1.00      0.99      1.00       617

    accuracy                           1.00      1234
   macro avg       1.00      1.00      1.00      1234
weighted avg       1.00      1.00      1.00      1234

[[617   0]
 [  6 611]]


In [14]:
model2 = Sequential()
model2.add(Conv2D(4, (3, 3), activation='relu',padding='same', input_shape=(200, 200, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 200, 200, 4)       112       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 100, 100, 4)      0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 100, 100, 16)      592       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 50, 50, 16)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 50, 50, 32)        4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 25, 25, 32)      

In [15]:
change_lr_callback = CustomCallbackReduceLr2()
history = model2.fit(x_train, y_train, batch_size=10, epochs=100, verbose=1, validation_split=0.1, callbacks = [change_lr_callback])

Epoch 1/100
111/111 [==============================] - 8s 68ms/step - loss: 0.3730 - accuracy: 0.8261 - val_loss: 0.1793 - val_accuracy: 0.9032
Epoch 2/100
111/111 [==============================] - 7s 67ms/step - loss: 0.1496 - accuracy: 0.9477 - val_loss: 0.2765 - val_accuracy: 0.9113
Epoch 3/100
111/111 [==============================] - 8s 73ms/step - loss: 0.0992 - accuracy: 0.9595 - val_loss: 0.0201 - val_accuracy: 0.9919
Epoch 4/100
111/111 [==============================] - 9s 79ms/step - loss: 0.0988 - accuracy: 0.9640 - val_loss: 0.1340 - val_accuracy: 0.9435
Epoch 5/100
111/111 [==============================] - 9s 80ms/step - loss: 0.0742 - accuracy: 0.9730 - val_loss: 0.0619 - val_accuracy: 0.9516
Epoch 6/100
111/111 [==============================] - 9s 80ms/step - loss: 0.0528 - accuracy: 0.9793 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 7/100
111/111 [==============================] - 9s 82ms/step - loss: 0.0644 - accuracy: 0.9775 - val_loss: 0.1069 - val_accuracy:

Epoch 30/100
111/111 [==============================] - 9s 82ms/step - loss: 1.9464e-04 - accuracy: 1.0000 - val_loss: 0.1271 - val_accuracy: 0.9919
Epoch 31/100
111/111 [==============================] - 9s 82ms/step - loss: 1.8464e-04 - accuracy: 1.0000 - val_loss: 0.1216 - val_accuracy: 0.9919
Epoch 32/100
111/111 [==============================] - 9s 82ms/step - loss: 1.6959e-04 - accuracy: 1.0000 - val_loss: 0.1284 - val_accuracy: 0.9919
Epoch 33/100
111/111 [==============================] - 9s 82ms/step - loss: 1.5874e-04 - accuracy: 1.0000 - val_loss: 0.1315 - val_accuracy: 0.9919
Changing Learning Rate to  0.0001250000059371814
Epoch 34/100
111/111 [==============================] - 9s 82ms/step - loss: 1.4451e-04 - accuracy: 1.0000 - val_loss: 0.1302 - val_accuracy: 0.9919
Epoch 35/100
111/111 [==============================] - 9s 82ms/step - loss: 1.3999e-04 - accuracy: 1.0000 - val_loss: 0.1292 - val_accuracy: 0.9919
Epoch 36/100
111/111 [==============================] - 9

Epoch 58/100
111/111 [==============================] - 9s 82ms/step - loss: 6.7695e-05 - accuracy: 1.0000 - val_loss: 0.1406 - val_accuracy: 0.9919
Epoch 59/100
111/111 [==============================] - 9s 81ms/step - loss: 6.6565e-05 - accuracy: 1.0000 - val_loss: 0.1410 - val_accuracy: 0.9919
Epoch 60/100
111/111 [==============================] - 9s 85ms/step - loss: 6.5381e-05 - accuracy: 1.0000 - val_loss: 0.1408 - val_accuracy: 0.9919
Epoch 61/100
111/111 [==============================] - 10s 89ms/step - loss: 6.3818e-05 - accuracy: 1.0000 - val_loss: 0.1418 - val_accuracy: 0.9919
Epoch 62/100
111/111 [==============================] - 12s 106ms/step - loss: 6.2098e-05 - accuracy: 1.0000 - val_loss: 0.1422 - val_accuracy: 0.9919
Epoch 63/100
111/111 [==============================] - 10s 94ms/step - loss: 6.0980e-05 - accuracy: 1.0000 - val_loss: 0.1431 - val_accuracy: 0.9919
Changing Learning Rate to  1.5625000742147677e-05
Epoch 64/100
111/111 [==============================

Epoch 86/100
111/111 [==============================] - 10s 94ms/step - loss: 4.4588e-05 - accuracy: 1.0000 - val_loss: 0.1465 - val_accuracy: 0.9919
Epoch 87/100
111/111 [==============================] - 10s 91ms/step - loss: 4.4342e-05 - accuracy: 1.0000 - val_loss: 0.1465 - val_accuracy: 0.9919
Epoch 88/100
111/111 [==============================] - 9s 82ms/step - loss: 4.3998e-05 - accuracy: 1.0000 - val_loss: 0.1471 - val_accuracy: 0.9919
Epoch 89/100
111/111 [==============================] - 9s 82ms/step - loss: 4.3710e-05 - accuracy: 1.0000 - val_loss: 0.1470 - val_accuracy: 0.9919
Epoch 90/100
111/111 [==============================] - 9s 82ms/step - loss: 4.3231e-05 - accuracy: 1.0000 - val_loss: 0.1473 - val_accuracy: 0.9919
Epoch 91/100
111/111 [==============================] - 9s 81ms/step - loss: 4.2991e-05 - accuracy: 1.0000 - val_loss: 0.1481 - val_accuracy: 0.9919
Epoch 92/100
111/111 [==============================] - 9s 83ms/step - loss: 4.2631e-05 - accuracy: 1.00

In [22]:
y_predicted = []
for i in range(len(y_test)):
    out_probabilities = model2.predict(x_test[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    
a = model2.evaluate(x_test, y_test)
print(classification_report(y_test, y_predicted))
print(confusion_matrix(y_test, y_predicted))

7/7 [==============================] - 1s 81ms/step - loss: 0.2356 - accuracy: 0.9587
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       109
           1       0.97      0.97      0.97       109

    accuracy                           0.97       218
   macro avg       0.97      0.97      0.97       218
weighted avg       0.97      0.97      0.97       218

[[106   3]
 [  3 106]]


In [23]:
y_predicted = []
for i in range(len(y_train)):
    out_probabilities = model2.predict(x_train[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)

a = model.evaluate(x_train, y_train)
print(classification_report(y_train, y_predicted))
print(confusion_matrix(y_train, y_predicted))

39/39 [==============================] - 3s 80ms/step - loss: 0.0298 - accuracy: 0.9951
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       617
           1       1.00      1.00      1.00       617

    accuracy                           1.00      1234
   macro avg       1.00      1.00      1.00      1234
weighted avg       1.00      1.00      1.00      1234

[[617   0]
 [  1 616]]


In [7]:
pruebas_root = r'kaggle_dataset'
x_pruebas, y_pruebas = load_images(pruebas_root)

In [26]:
y_predicted = []
for i in range(len(y_pruebas)):
    out_probabilities = model.predict(x_pruebas[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    
a = model.evaluate(x_pruebas, y_pruebas)
print(classification_report(y_pruebas, y_predicted))
print(confusion_matrix(y_pruebas, y_predicted))

125/125 [==============================] - 12s 100ms/step - loss: 0.2250 - accuracy: 0.9470
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2000
           1       0.96      0.93      0.95      2000

    accuracy                           0.95      4000
   macro avg       0.95      0.95      0.95      4000
weighted avg       0.95      0.95      0.95      4000

[[1921   79]
 [ 133 1867]]


In [27]:
y_predicted = []
for i in range(len(y_pruebas)):
    out_probabilities = model2.predict(x_pruebas[i:i+1])
    number_detected = 1 if out_probabilities[0][0] > 0.5 else 0
    y_predicted.append(number_detected)
    
a = model2.evaluate(x_pruebas, y_pruebas)
print(classification_report(y_pruebas, y_predicted))
print(confusion_matrix(y_pruebas, y_predicted))

125/125 [==============================] - 12s 96ms/step - loss: 0.2250 - accuracy: 0.9470
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      2000
           1       0.97      0.84      0.90      2000

    accuracy                           0.91      4000
   macro avg       0.91      0.91      0.91      4000
weighted avg       0.91      0.91      0.91      4000

[[1946   54]
 [ 320 1680]]


In [34]:
model.save("model/eye_classifier1.h5")
model2.save("model/eye_classifier2.h5")
print("Saved model to disk")

Saved model to disk
